In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random

print("===INFO===")
print("torch ver : %s\ntorchvision ver : %s " %(torch.__version__, torchvision.__version__))
print("GPU model :",torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

===INFO===
torch ver : 1.8.0
torchvision ver : 0.2.2 
GPU model : TITAN RTX


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#error_index = 0
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)


In [3]:
from importnb import Notebook

In [4]:
with Notebook(lazy=True):
    import F4F_model
header = F4F_model

In [5]:
# set randomness
seed = 0

header.set_randomness(seed) # ipynb module import means run all cell in file
# load dataset
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0722_to_fullmodels/"
batch_size = 32 # 32~ out of memory in 3080
num_train = 128

train_dataloader,test_dataloader = header.get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

===INFO===
torch ver : 1.8.0
torchvision ver : 0.2.2 
GPU model : TITAN RTX
anaconda3	   environment.yml  jupyter.sh	todolist
docker_script.txt  F4F		    pastebin
train dataset[4], test dataset[1563] are loaded


In [6]:
#In_layer_number = 34 # 34 conv5_1 convolution
#Out_layer_number = 36 # 36 conv5_1 relu 
error_index=0
max_epochs = 30
num_error = 128

In [7]:
log_file = "./128_acc_log_toEnd_Debug.txt"

In [8]:
#split_model = split_layer(vgg16_bn,0,Out_layer_number)

original_model = d_copy(vgg16_bn).to(device)
# subset of vgg16 (whole layer) with f4f
header.hook_register(vgg16_bn,error_index,num_error)
target_model = header.Target_model(vgg16_bn).to(device)



In [9]:
f4f = header.F4F().to(device)
optimizer = torch.optim.SGD(f4f.parameters(),lr=0.1e-5,weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()


In [49]:

def make_error_info(error_index, num_error):
    data = []
    #for i in range(511,-1,-1): 0,0,0,1
    for i in range(512):
        if error_index <= i and i < error_index+num_error:
            data.append(1)
        else :
            data.append(0)
        #print(data)
    error_info = torch.Tensor(data)
    print(error_info)
    error_info  = error_info.unsqueeze(0).repeat(512,1)
    return error_info # 512,521

In [50]:
error_info = make_error_info(0,num_error).to(device)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [11]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)

f = open(log_file,"w")
f.close()
num_error = 128
max_epoch = 50


In [12]:
# training
def training(f4f,target_model,original_model,
             train_dataloader,test_dataloader,
             log_file,loss_fn,optimizer,
             error_idx,num_error,
             max_epochs=30,subset=False):
    first_feature = []
    original_out = []
    first_label = []
    target_model.to(device)
    original_model.to(device)
    error_info = make_error_info(error_idx,num_error).to(device)
    first_feature = []
    
    feature_num = 100
    for epoch in range(max_epochs):
        running_loss = 0.0
        total_loss = []
        total = 0
        correct = 0
        f4f.train()
        # update f4f filter
        #target_model.apply_f4f(f4f,error_info)
    
        # compare
        for i, data in enumerate(train_dataloader):
            if i % 10 == 0:
                print(".",end="")
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
            
            target_out = target_model(inputs,f4f,error_info)
            #print(original_out[0][0][0],target_out[0][0][0])
            #exit(0)
            
            first_feature.append(target_out)
            first_label.append(labels)
            if len(first_feature) > feature_num:
                first_feature.pop(0)
                first_label.pop(0)
            _,predicted = torch.max(target_out,1) # target_out.data : no grad, target_out : with grad
            
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
            #print(labels.size(),target_out.size())
            loss = loss_fn(target_out,labels)
            
            running_loss += loss.item()
            target_model.model.zero_grad() # might be useless
            f4f.f4f.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 99: 
                total_loss.append(running_loss/100)
                print("")
                print('[%d, %5d] loss: %.6f' % (epoch+1, i+1, running_loss/100)) 
                running_loss = 0.0
        # save weight
        #print((len(train_dataloader)/batch_size))
        #total_avg_loss = sum(total_loss)/len(total_loss)
        acc = 100*correct/total
        #print("total average loss : %.3f" %(total_avg_loss))
        print("train acc : %.4f" %(acc))
        #acc = header.eval(target_model,test_dataloader,epoch,f4f,error_info,log_file)
        """
        torch.save(f4f.get_f4f_weight(), 
               retrain_model_path+"%s~%s_pkt_err_f4f_epoch_%s_acc_%.4f_loss_%.4f.pt"
               %(str(error_idx).zfill(3),str(error_idx+num_error).zfill(3),
                str(epoch+1).zfill(2),acc,total_avg_loss))    
        """
        return first_feature,first_label
                

In [13]:
first_feature,first_label = training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,
                  log_file, loss_fn,optimizer,
                  error_index,num_error,10,True)

.torch.Size([512, 4608])
torch.Size([512, 512, 3, 3])
torch.Size([512, 4608])
torch.Size([512, 512, 3, 3])
torch.Size([512, 4608])
torch.Size([512, 512, 3, 3])
torch.Size([512, 4608])
torch.Size([512, 512, 3, 3])
train acc : 0.0000


In [14]:
len(first_feature[0]),first_feature[0].size()

(32, torch.Size([32, 1000]))

In [15]:
print(first_feature[0].size(),first_label[0].size())
#torch.max(mp[1][0],1) // 32,1000

for i in range(first_label[0].size()[0]):
    print(first_feature[0][i].argmax().item(),"vs",first_label[0][i].item())

torch.Size([32, 1000]) torch.Size([32])
491 vs 477
696 vs 903
836 vs 594
674 vs 994
539 vs 707
836 vs 95
428 vs 818
674 vs 687
456 vs 194
852 vs 334
491 vs 445
971 vs 239
971 vs 285
876 vs 555
487 vs 997
491 vs 482
428 vs 177
921 vs 497
691 vs 321
691 vs 666
882 vs 355
112 vs 89
674 vs 304
539 vs 975
917 vs 665
696 vs 513
787 vs 883
428 vs 58
790 vs 837
589 vs 736
447 vs 765
851 vs 702


In [16]:
# 여기서부터는 feature 그림 보기 위한 것들입니다.


In [17]:

w = 10
h = 10
cols = 32
rows = 16
def feature_print(pic):
    print("test with 'after pooling 4 feature'")
    fig = plt.figure(figsize=(64,32))
    ax = []
    for i in range(cols*rows):
        ch = pic[i,:,:]
        ax.append(fig.add_subplot(rows,cols,i+1))
        ax[-1].set_title(str(i)+"th ch (14x14)")
        plt.imshow(ch)